*Clustering Analysis*

An Italian wine consortium representing vineyards in a specific region has commissioned a study to improve its understanding of
the chemical properties of its wines. The region produces three distinct wine varieties (cultivars), and the consortium wants to
ensure that each cultivar maintains its unique quality and consistency while exploring new ways to market its products effectively.

You are hired as a data analyst to help the consortium achieve these goals. Your task is to use K-Means clustering to group the
wines based on their chemical properties and identify natural clusters.

The dataset is named "wine_clustering.csv" and can be downloaded from the “Project Datasets” folder on myLMS

Implement K-means clustering on the data in the scenario above. Ensure that you standardise your data. You may use either the
elbow or silhouette method to determine the optimal number of clusters. After implementing the clustering, create a plot to
visualise the clusters you have created.
(10 marks)